# SmoothQuant on OPT-13B

### Guangxuan Xiao*, Ji Lin*, Mickael Seznec, Julien Demouth, Song Han

In this notebook, we use OPT-13B model to demonstrate SmoothQuant can use 8-bit for both weights and activations to achieve the same accuracy as FP16 models. Unlike previous method [[Dettmers *et al.*, 2022]](https://arxiv.org/abs/2208.07339), SmoothQuant enables fully INT8 GEMMs for linear layers and does not require high precision numbers to represent outliers. We have tested SmoothQuant on up to 176 billion parameter models (BLOOM).

In order to run this notebook, you need to install the following packages:

- smoothquant
- PyTorch
- Transformers
- Accelerate

In [1]:
import torch
from transformers.models.opt.modeling_opt import OPTAttention, OPTDecoderLayer, OPTForCausalLM
from transformers import GPT2Tokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import W8A8Linear

In this notebook, we simulate the 8-bit dynamic per-tensor weight and activation quantization with FP16, i.e., fake quantization. We have implmented the real 8-bit quantization with INT8 CUTLASS GEMM kernels for both PyTorch and FasterTransformer. Please stay tuned for the release.

In [2]:
def quantize_model(model, weight_quant='per_tensor', act_quant='per_tensor', quantize_bmm_input=True):
    for name, m in model.model.named_modules():
        if isinstance(m, OPTDecoderLayer):
            m.fc1 = W8A8Linear.from_float(m.fc1, weight_quant=weight_quant, act_quant=act_quant)
            m.fc2 = W8A8Linear.from_float(m.fc2, weight_quant=weight_quant, act_quant=act_quant)
        elif isinstance(m, OPTAttention):
            # Her we simulate quantizing BMM inputs by quantizing the output of q_proj, k_proj, v_proj
            m.q_proj = W8A8Linear.from_float(
                m.q_proj, weight_quant=weight_quant, act_quant=act_quant, quantize_output=quantize_bmm_input)
            m.k_proj = W8A8Linear.from_float(
                m.k_proj, weight_quant=weight_quant, act_quant=act_quant, quantize_output=quantize_bmm_input)
            m.v_proj = W8A8Linear.from_float(
                m.v_proj, weight_quant=weight_quant, act_quant=act_quant, quantize_output=quantize_bmm_input)
            m.out_proj = W8A8Linear.from_float(m.out_proj, weight_quant=weight_quant, act_quant=act_quant)
    return model


The following is an evaluator to see the performance of the model. We use a toy dataset (the first 1000 examples in the validation set of the Lambada dataset) to evaluate the model. You can replace it with your own dataset. The conclusion should be the same.

In [3]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        # tokenize the dataset
        def tokenize_function(examples):
            example = self.tokenizer(examples['text'])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type='torch', columns=['input_ids'])

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        # The task is to predict the last word of the input.
        total, hit = 0, 0
        for batch in self.dataset:
            input_ids = batch['input_ids'].to(self.device).unsqueeze(0)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            last_token_logits = outputs.logits[:, -2, :]
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
        acc = hit / total
        return acc


/home/guangxuan/anaconda3/envs/smoothquant/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
from datasets import load_dataset

tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-13b')
dataset = load_dataset('lambada', split='validation[:1000]')
evaluator = Evaluator(dataset, tokenizer, 'cuda')


Reusing dataset lambada (/home/guangxuan/.cache/huggingface/datasets/lambada/plain_text/1.1.0/e32d76a7236c9ebb30099bc73d677c3acf32ddffb411836fe9ffc091ad3f3bec)
Parameter 'function'=<function Evaluator.__init__.<locals>.tokenize_function at 0x7ff2a44f3700> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

Let's first check the performance of the original FP16 model.

In [5]:
model_fp16 = OPTForCausalLM.from_pretrained('facebook/opt-13b', torch_dtype=torch.float16, device_map='auto')

In [6]:
acc_fp16 = evaluator.evaluate(model_fp16)
print(f'Original model (fp16) accuracy: {acc_fp16}')


Original model (fp16) accuracy: 0.786


We then quantize the model to W8A8 and check the performance.

In [7]:
model_w8a8 = quantize_model(model_fp16)
print(model_w8a8)


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 5120, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 5120)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_aff

In [8]:
acc_w8a8 = evaluator.evaluate(model_w8a8)
print(f'Naive W8A8 quantized model accuracy: {acc_w8a8}')

Naive W8A8 quantized model accuracy: 0.048


We can see there is a significant accuracy drop. This is consistent with LLM.int8()'s finding: when the model size increases larger than 6.7B, systematic outliers will emerge in activations, which makes fully INT8 quantization impossible.

Let's smooth the model, quantize it, and check the performance! In `../act_scales`, we provide the activation scales for OPT and BLOOM models. You can also use this notebook to test quantizing those models.

In [9]:
model = OPTForCausalLM.from_pretrained('facebook/opt-13b', torch_dtype=torch.float16, device_map='auto')
act_scales = torch.load('../act_scales/opt-13b.pt')
smooth_lm(model, act_scales, 0.5)
model_smoothquant_w8a8 = quantize_model(model)
print(model_smoothquant_w8a8)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 5120, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 5120)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(5120, 5120, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_aff

We can see the smoothed model has the same accuracy as the FP16 model. This is because SmoothQuant smooths the outliers in activations and moves the quantization difficulty from activations to weights.

In [10]:
acc_smoothquant_w8a8 = evaluator.evaluate(model_smoothquant_w8a8)
print(f'SmoothQuant W8A8 quantized model accuracy: {acc_smoothquant_w8a8}')

SmoothQuant W8A8 quantized model accuracy: 0.793
